In [ ]:
import torch
import numpy as np
from glob import glob
import os
PATH = '/home/michael/Master-Thesis/CKIM_Competition/exp/Feature_Analysis_Graph-DC_FedVAE_2_out_global_private_NEW_sim_loss_lr_0_1_A0_F0_001_G0_001_H10_I5sim_loss_mse/FedDomSep_GraphDC_gin_on_fs_contest_data_lr0.1_lstep1_/features'

In [ ]:
interm_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
global_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
local_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
labels = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}


In [ ]:
all_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, '*.pt'))]
all_files.sort()

In [ ]:
def get_files(path):
    for client in range(1, 13+1):
        client_path = f"{PATH}/client_{client}"
        all_files = [file
                 for path, subdir, files in os.walk(client_path)
                 for file in glob(os.path.join(path, '*.pt'))]
        all_files.sort()
        for file in all_files:
            file_name = os.path.basename(file)
            file_name_without_ending = file_name[:-3]
            if file_name_without_ending.startswith('global_val') and not file_name_without_ending.endswith('labels'):
                global_trains[client].append(torch.load(file).cpu().detach().numpy())
            if file_name_without_ending.startswith('interm_val') and not file_name_without_ending.endswith('labels'):
                interm_trains[client].append(torch.load(file).cpu().detach().numpy())
            if file_name_without_ending.startswith('local_val') and not file_name_without_ending.endswith('labels'):
                local_trains[client].append(torch.load(file).cpu().detach().numpy())
            if file_name_without_ending.startswith('val') and file_name_without_ending.endswith('labels'):
                labels[client].append(torch.load(file).cpu().detach().numpy())
        interm_trains[client] = np.concatenate(interm_trains[client])
        global_trains[client] = np.concatenate(global_trains[client])
        local_trains[client] = np.concatenate(local_trains[client])
        labels[client] = np.concatenate(labels[client])


In [ ]:
get_files(PATH)

In [ ]:
import numpy as np
#concat_X=global_train
data_out_global = {}
data_out_interm = {}

for key in interm_trains.keys():
    data_out_global[key] = global_trains[key] + local_trains[key]
    data_out_interm[key] = interm_trains[key] + local_trains[key]

X_concat_global = []
X_concat_interm = []
labels_concat = []

for key in interm_trains.keys():
    X_concat_global.append(data_out_global[key])
    X_concat_interm.append(data_out_interm[key])
    labels_concat.append(labels[key])

X_concat_global = np.concatenate(X_concat_global)
X_concat_interm = np.concatenate(X_concat_interm)
labels_concat = np.concatenate(labels_concat)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
labels_global = labels.copy()


In [ ]:
X_concat = np.concatenate([X_concat_global, X_concat_interm])
labels_double_concat = np.concatenate([labels_concat])

In [ ]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=25).fit_transform(X_concat, labels_double_concat)

In [ ]:
X_embedded.shape[0]//2

In [ ]:
# Feature Analysis for client 9:
global_trains[9]

In [ ]:
interm_trains[9]

In [ ]:
local_trains[9]

In [ ]:
cos_sim = torch.nn.CosineSimilarity()

In [ ]:
for client in range(1, 13+1):


    local_train_curr = local_trains[client]
    interm_train_curr = interm_trains[client]
    global_train_curr = global_trains[client]

    num_total_features = local_train_curr.shape[0] * local_train_curr.shape[1]

    local_interm = local_train_curr + interm_train_curr
    local_global = local_train_curr + global_train_curr

    mult_local_interm = local_train_curr*interm_train_curr
    mult_local_global = global_train_curr*local_train_curr
    mult_global_interm = interm_train_curr*global_train_curr

    num_local_features_not_0 =  np.sum(local_train_curr != 0) / num_total_features
    avg_local_features_not_0 =  local_train_curr.sum() / np.sum(local_train_curr != 0)
    num_global_features_not_0 =  np.sum(global_train_curr != 0) / num_total_features
    avg_global_features_not_0 =  global_train_curr.sum() / np.sum(global_train_curr != 0)
    num_interm_features_not_0 =  np.sum(interm_train_curr != 0) / num_total_features
    avg_interm_features_not_0 =  interm_train_curr.sum() / np.sum(interm_train_curr != 0)

    num_features_global_local = np.sum(mult_local_global != 0) / num_total_features
    num_features_interm_local = np.sum(mult_local_interm != 0) / num_total_features
    num_features_global_interm = np.sum(mult_global_interm != 0) / num_total_features


    cos_sim_global_interm = cos_sim(torch.Tensor(global_train_curr), torch.Tensor(interm_train_curr)).mean()
    cos_sim_local_local_interm = cos_sim(torch.Tensor(local_train_curr), torch.Tensor(local_interm)).mean()
    cos_sim_local_local_global = cos_sim(torch.Tensor(local_train_curr), torch.Tensor(local_global)).mean()

    print(f"############ Client {client} ##############")
    print(f"total features: {num_total_features}")
    print(f"num_local_features_not_0: {num_local_features_not_0}")
    print(f"avg_local_features_not_0: {avg_local_features_not_0}")

    print(f"num_interm_features_not_0: {num_interm_features_not_0}")
    print(f"num_features_interm_local: {num_features_interm_local}")
    print(f"avg_interm_features_not_0: {avg_interm_features_not_0}")

    print(f"num_global_features_not_0: {num_global_features_not_0}")
    print(f"num_features_global_local: {num_features_global_local}")
    print(f"num_features_global_interm: {num_features_global_interm}")
    print(f"avg_global_features_not_0: {avg_global_features_not_0}")

    print(f"cos_sim_global_interm: {cos_sim_global_interm}")
    print(f"cos_sim_local_local_interm: {cos_sim_local_local_interm}")
    print(f"cos_sim_local_local_global: {cos_sim_local_local_global}")


In [ ]:
local_trains[9].sum() / np.sum(local_trains[9] != 0)

In [ ]:
np.sum(local_trains[9] != 0)

In [ ]:
np.sum(interm_trains[9] != 0)

In [ ]:
interm_trains[9].sum() / np.sum(interm_trains[9] != 0)

In [ ]:
global_trains[9].sum() / np.sum(global_trains[9] != 0)

In [ ]:
np.sum(global_trains[9] != 0)

In [ ]:
global_trains[9].shape[0] * global_trains[9].shape[1]

In [ ]:
X_embedded_global = X_embedded[:3883,:]
X_embedded_interm = X_embedded[3883:,:]

In [ ]:
#X_embedded_global = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=25).fit_transform(X_concat_global, labels_concat)
#X_embedded_local = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=25).fit_transform(X_concat_interm, labels_concat)
#X_embedded_interm = X_embedded_local

In [ ]:
#X_embedded_interm = X_embedded_local

In [ ]:
X_embedded_global_clients = {}
X_embedded_interm_clients = {}
labels_clients = {}
total_size = 0

for client in range(1, 13+1):
    print(f"size client {client}: {interm_trains[client].shape[0]}")
    size = interm_trains[client].shape[0]
    size_old = total_size
    total_size += size
    X_embedded_global_clients[client] = X_embedded_global[size_old:total_size,:]
    X_embedded_interm_clients[client] = X_embedded_interm[size_old:total_size,:]
    labels_clients[client] = labels_concat[size_old:total_size]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30, 18))
plt.rcParams["figure.figsize"] = (30,18)

In [ ]:
"""yellow: label=1"""

In [ ]:
client_num = 13

In [ ]:
plt.scatter(X_embedded_global_clients[client_num][:, 0], X_embedded_global_clients[client_num][:, 1], c=labels_clients[client_num], )
plt.xlim(-110, 110)
plt.ylim(-100, 90)
plt.show()

In [ ]:
plt.scatter(X_embedded_interm_clients[client_num][:, 0], X_embedded_interm_clients[client_num][:, 1], c=labels_clients[client_num], )
plt.xlim(-110, 110)
plt.ylim(-100, 90)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30, 18))
plt.rcParams["figure.figsize"] = (30,18)

In [ ]:
plt.scatter(X_embedded_global[:, 0], X_embedded_global[:, 1], c=labels_concat)
plt.show()

In [ ]:

plt.scatter(X_embedded_interm[:, 0], X_embedded_interm[:, 1], c=labels_concat)
plt.show()

In [ ]:
color_map = {0:'red', 1:'yellow', 2:'orange', 3:'green', 4:'cyan', 5:'grey'}
colors = list(map(color_map.get, labels))

In [ ]:
scatter = ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=colors)

In [ ]:
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="lower left", title="Classes")

In [ ]:
ax.add_artist(legend1)

In [ ]:
plt.show()

In [ ]:
fig

In [ ]:
scatter.legend_elements()

In [ ]:
X_embedded.shape[0]

In [ ]:
X_embedded.shape[0]//2

In [ ]:
X_embedded_combined = X_embedded[:X_embedded.shape[0]//2, :]

In [ ]:
X_embedded_combined

In [ ]:
X_embedded_local = X_embedded[X_embedded.shape[0]//2:, :]

In [ ]:
X_embedded_local

In [ ]:
x_diff = X_embedded_local - X_embedded_combined

In [ ]:
x_diff

In [ ]:
labels